Dask and Scikit-Image on a Cluster
==================================

<img src="http://scikit-image.org/_static/img/logo.png">

In [ ]:
%matplotlib inline
from dask import delayed
import skimage.io
import matplotlib.pyplot as plt

In [ ]:
sample = skimage.io.imread('https://storage.googleapis.com/pangeo-data/FIB-25/iso.00000.png')

fig = plt.figure(figsize=(10, 10))
plt.imshow(sample, cmap='gray')

In [ ]:
from dask import delayed
import dask.array as da

# filenames = ['https://storage.googleapis.com/pangeo-data/FIB-25/iso.%05d.png' % i for i in range(0, 8090)]
filenames = ['https://storage.googleapis.com/pangeo-data/FIB-25/iso.%05d.png' % i for i in range(0, 1000)]
images = [delayed(skimage.io.imread)(fn) for fn in filenames]
arrays = [da.from_delayed(im, shape=sample.shape, dtype=sample.dtype) for im in images]
x = da.stack(arrays, axis=0)
x

In [ ]:
x.nbytes / 1e9

In [ ]:
from dask_kubernetes import KubeCluster
cluster = KubeCluster()
cluster

In [ ]:
from dask.distributed import Client, progress
client = Client(cluster)
client

In [ ]:
y = x.rechunk((200, 200, 200))
y = y.persist()
progress(y)

In [ ]:
y

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(y[500], cmap='gray')

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(y[:, 3000, 3000])

In [ ]:
import skimage

y.map_overlap?
